In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.utils import image_dataset_from_directory

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

2.10.1
Num GPUs Available:  1


Data Preprocessing 

In [19]:
training_data = "data/Training"
testing_data = "data/Testing"

class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

def normalize_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_generator = image_dataset_from_directory(
    training_data,
    subset='training',
    seed=50,
    image_size=(256, 256),
    batch_size=64,
    validation_split=0.1,
).map(normalize_image)

validation_generator = image_dataset_from_directory(
    training_data,
    subset='validation',
    seed=50,
    image_size=(256, 256),
    batch_size=64,
    validation_split=0.3,
).map(normalize_image)

test_generator = image_dataset_from_directory(
    testing_data,
    image_size=(256, 256),
    batch_size=64,
    seed=50
).map(normalize_image)


Found 5712 files belonging to 4 classes.
Using 3999 files for training.
Found 5712 files belonging to 4 classes.
Using 2284 files for validation.
Found 1311 files belonging to 4 classes.


Show graph

In [32]:

# Code for plotting
# def show(images, labels):
#     plt.figure(figsize=(10, 10))
#     for i in range(16):
#         plt.subplot(4, 4, i + 1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.grid(False)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.xlabel(class_names[labels[i]])
#     plt.show()

# show(train_images, train_labels)

Model

In [20]:
from tensorflow.keras.optimizers import Adam
model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4, activation='softmax'))
print(model.summary())


model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

batch_size = 64
epochs = 10

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 128, 128, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 32, 32, 64)      

Fit

In [21]:
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
)


Epoch 1/10
63/63 [==============================] - 4s 61ms/step - loss: 0.8070 - accuracy: 0.6729 - val_loss: 0.5013 - val_accuracy: 0.8008
Epoch 2/10
63/63 [==============================] - 3s 52ms/step - loss: 0.4820 - accuracy: 0.8130 - val_loss: 0.4155 - val_accuracy: 0.8275
Epoch 3/10
63/63 [==============================] - 3s 52ms/step - loss: 0.3298 - accuracy: 0.8747 - val_loss: 0.3118 - val_accuracy: 0.8905
Epoch 4/10
63/63 [==============================] - 3s 53ms/step - loss: 0.2839 - accuracy: 0.8940 - val_loss: 0.2393 - val_accuracy: 0.9199
Epoch 5/10
63/63 [==============================] - 3s 52ms/step - loss: 0.2180 - accuracy: 0.9202 - val_loss: 0.2147 - val_accuracy: 0.9387
Epoch 6/10
63/63 [==============================] - 3s 53ms/step - loss: 0.1514 - accuracy: 0.9432 - val_loss: 0.1980 - val_accuracy: 0.9418
Epoch 7/10
63/63 [==============================] - 3s 53ms/step - loss: 0.1076 - accuracy: 0.9645 - val_loss: 0.1661 - val_accuracy: 0.9514
Epoch 8/10
63

Evaulation

In [22]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.2f}')
print(f'Test loss: {test_loss:.2f}')

21/21 [==============================] - 1s 13ms/step - loss: 0.2838 - accuracy: 0.9306
Test accuracy: 0.93
Test loss: 0.28
